In [1]:
from rbatools.rba_Session import RBA_Session
import matplotlib.pyplot as plt
import pandas

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.8.3 64-bit ('base': conda).

# Load E.coli model 

In [ ]:
model_filename = 'Bacterial-RBA-models/Escherichia-coli-K12-WT'
Simulation = RBA_Session(model_filename)


Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.8.3 64-bit ('base': conda).

# Exporting a tabular represention for the structure of the model in the SBtab-format, for inspection in Excel etc...

In [ ]:
Simulation.ModelStructure.exportSBtab(filename='Structure_Ecoli_RBA_Model')

# Setting medium, finding maximum growth-rate and handle simulation-results

## Perform growth-rate optimisation for different glucose-concentrations and record the corresponding simulation-results

In [ ]:
Simulation.setMedium({'M_glc__D':0.2})
Simulation.findMaxGrowthRate()
Simulation.recordResults('Glc_02')

Simulation.setMedium({'M_glc__D':0.4})
Simulation.findMaxGrowthRate()
Simulation.recordResults('Glc_04')

Simulation.setMedium({'M_glc__D':0.6})
Simulation.findMaxGrowthRate()
Simulation.recordResults('Glc_06')

Simulation.setMedium({'M_glc__D':0.8})
Simulation.findMaxGrowthRate()
Simulation.recordResults('Glc_08')

Simulation.setMedium({'M_glc__D':1})
Simulation.findMaxGrowthRate()
Simulation.recordResults('Glc_1')

Simulation.writeResults()



## Export Results as single SBtab-file

In [ ]:
Simulation.SimulationData.exportSBtab(filename_SBtab='Ecoli_GlucoseScreen')

## Export flux-distribution for each run (glucose level) as input-file for Escher-map visualisation on https://escher.github.io

In [ ]:
Simulation.SimulationData.exportEscherMap()

## Export proteome for each run (glucose level) as input-file for Proteo-map visualisation on https://www.proteomaps.net

In [ ]:
Simulation.SimulationData.exportProteoMap()

# Performing variability-analysis for glycolysis-reaction at different growth-rates

## Define medium and determine maximum-growth-rate

In [ ]:
Simulation.setMedium({'M_glc__D':2})
mumax=Simulation.findMaxGrowthRate()


## Determine the feasible-range of the glycolytic Phosphoglycerate kinase at different fractions of the maximum growth-rate

In [ ]:
Simulation.setMu(mumax*0.5)
FR_PGK_05=Simulation.FeasibleRange(variables='R_PGK')

Simulation.setMu(mumax*0.6)
FR_PGK_06=Simulation.FeasibleRange(variables='R_PGK')

Simulation.setMu(mumax*0.7)
FR_PGK_07=Simulation.FeasibleRange(variables='R_PGK')

Simulation.setMu(mumax*0.8)
FR_PGK_08=Simulation.FeasibleRange(variables='R_PGK')


Simulation.setMu(mumax*0.9)
FR_PGK_09=Simulation.FeasibleRange(variables='R_PGK')

Simulation.setMu(mumax*0.999999)
FR_PGK_0999999=Simulation.FeasibleRange(variables='R_PGK')



## Plot the evolution of the feasible-range at different growth-rates

In [ ]:
Mins=[FR_PGK_05['R_PGK']['Min'],FR_PGK_06['R_PGK']['Min'],FR_PGK_07['R_PGK']['Min'],FR_PGK_08['R_PGK']['Min'],FR_PGK_09['R_PGK']['Min'],FR_PGK_0999999['R_PGK']['Min']]
Maxs=[FR_PGK_05['R_PGK']['Max'],FR_PGK_06['R_PGK']['Max'],FR_PGK_07['R_PGK']['Max'],FR_PGK_08['R_PGK']['Max'],FR_PGK_09['R_PGK']['Max'],FR_PGK_0999999['R_PGK']['Max']]
Mus=[mumax*0.5,mumax*0.6,mumax*0.7,mumax*0.8,mumax*0.9,mumax*0.999999]

plt.figure(figsize=(9,6))
plt.scatter(Mus,Mins,color='red')
plt.scatter(Mus,Maxs,color='blue')

plt.title('Feasible-range of PGK-reaction vs. growth-rate',fontsize=20)
plt.xlabel('Growth-rate',fontsize=15)
plt.ylabel('Flux',fontsize=15)
plt.legend(['Upper-limit of feasible-range','Lower-limit of feasible-range'])


# Determine the Pareto-front between two different glucose-uptake systems (PTS-system and proton-symport) for the wild-type and a knock-out of the upper Pentose-Phosphate-Pathway 

## Set growth-rate to 99,9999% of the maximum and determine the Pareto-front between the GLCptspp-reaction and the GLCt2pp-reaction

In [ ]:
Simulation.setMu(mumax*0.999999)
Pareto_Front_1=Simulation.ParetoFront(variable_X='R_GLCptspp', variable_Y='R_GLCt2pp', N=10, sign_VY='max')

## Display the output data-structure of the Pareto-front analysis

In [ ]:
print(Pareto_Front_1)

## I want to knock-out the G6PDH2r-reaction in the upper Pentose-Phosphate-Pathway. So I have to look up, which gene is responsible for constructing the respective enzyme 'R_G6PDH2r_enzyme'

In [ ]:
Simulation.ModelStructure.EnzymeInfo.Elements['R_G6PDH2r_enzyme']

## Knocking out the gene b1852, which makes up the enzyme of the G6PDH2r-reaction

In [ ]:
Simulation.knockOut(gene='b1852', loggingIntermediateSteps=False)


## Determining maximum growth-rate of knock-out strain

In [ ]:
mumax2=Simulation.findMaxGrowthRate()


## Set growth-rate to 99,9999% of the maximum and determine the Pareto-front between the GLCptspp-reaction and the GLCt2pp-reaction for the KO-strain

In [ ]:
Simulation.setMu(mumax2*0.999999)
Pareto_Front_2=Simulation.ParetoFront(variable_X='R_GLCptspp', variable_Y='R_GLCt2pp', N=10, sign_VY='max')

## Plot the Pareto-fronts for the WT- and KO-strain

In [ ]:
plt.figure(figsize=(9,6))
plt.scatter(Pareto_Front_1['R_GLCptspp'],Pareto_Front_1['R_GLCt2pp'],color='blue')
plt.scatter(Pareto_Front_2['R_GLCptspp'],Pareto_Front_2['R_GLCt2pp'],color='red')
plt.title('Pareto-front between two glucose-uptake systems',fontsize=20)
plt.xlabel('R_GLCptspp flux',fontsize=15)
plt.ylabel('R_GLCt2pp flux',fontsize=15)
plt.legend(['Wild-Type (Mu={})'.format(str(round(mumax*0.999999,2))),'Upper PPP-KO (Mu={})'.format(str(round(mumax2*0.999999,2)))])
plt.ylim(bottom=0)
plt.show()